In [ ]:
import numpy as np
from sklearn.decomposition import FastICA
from scipy.signal import butter, lfilter, correlate, detrend
from scipy.stats import entropy

# Function to apply bandpass filter
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data)
    return y

# Function to calculate negentropy
def calculate_negentropy(signal):
    signal_norm = (signal - np.mean(signal)) / np.std(signal)
    gaussian = np.random.normal(0, 1, len(signal))
    return entropy(gaussian) - entropy(signal_norm)

# Function to calculate autocorrelation
def calculate_autocorrelation(signal):
    result = correlate(signal, signal, mode='full')
    max_corr = np.max(result[result.size // 2:])
    return max_corr / len(signal)

# MAICA method for multi-objective optimization
def maica(ica_signals, fs):
    best_signal = None
    best_score = -np.inf

    for i in range(ica_signals.shape[1]):
        signal = ica_signals[:, i]
        filtered_signal = bandpass_filter(signal, 0.7, 4.0, fs)
        
        # Calculate metrics
        autocorr = calculate_autocorrelation(filtered_signal)
        negentropy = calculate_negentropy(filtered_signal)
        
        # Combine metrics (this is a simple weighted sum, adjust weights as needed)
        score = autocorr + negentropy
        
        if score > best_score:
            best_score = score
            best_signal = filtered_signal

    return best_signal

# Capture video
cap = cv2.VideoCapture(0)
fs = 30.0  # Frame rate
roi = (100, 100, 200, 200)  # Example ROI, adjust as needed
rgb_signals = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    r, g, b = extract_rgb_means(frame, roi)
    rgb_signals.append([r, g, b])
    
    cv2.rectangle(frame, (roi[0], roi[1]), (roi[2], roi[3]), (255, 0, 0), 2)
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

rgb_signals = np.array(rgb_signals)

# Detrend and normalize
rgb_signals = detrend(rgb_signals, axis=0)
rgb_signals = (rgb_signals - np.mean(rgb_signals, axis=0)) / np.std(rgb_signals, axis=0)

# Apply ICA
ica = FastICA(n_components=3)
ica_signals = ica.fit_transform(rgb_signals)

# Apply MAICA
rppg_signal = maica(ica_signals, fs)

# Estimate heart rate using FFT
n = len(rppg_signal)
t = np.arange(n) / fs
yf = fft(rppg_signal)
xf = fftfreq(n, 1 / fs)[:n // 2]

# Find the peak frequency and convert to bpm
peak_freq = xf[np.argmax(np.abs(yf[:n // 2]))]
heart_rate = peak_freq * 60  # Convert to bpm
print(f"Estimated Heart Rate: {heart_rate:.2f} bpm")

# Plot the rPPG signal and FFT
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(t, rppg_signal)
plt.title('rPPG Signal')
plt.subplot(2, 1, 2)
plt.plot(xf, 2.0 / n * np.abs(yf[:n // 2]))
plt.title('FFT of rPPG Signal')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude')
plt.show()
